# Análisis de Scoring Crediticio - Proyecto de Grado
## Framework de RL para Optimización de Datos Sintéticos y Modelos ML

**Título:** Sintetizando datos tabulares: Un Framework de Aprendizaje por Refuerzo para el Benchmark de Datos Sintéticos y su Impacto en problemas de Clasificación

**Autor:** Carlos Andres Cortez Ballen

### Objetivos del Proyecto:
- Implementar y comparar generadores de datos sintéticos (GANs vs métodos tradicionales)
- Construir framework de RL para selección óptima de generadores y modelos ML
- Evaluar rendimiento en scoring crediticio para segmento D (baja transaccionalidad)
- Desarrollar métricas de calidad para datos sintéticos
- Realizar benchmarking cruzado entre estrategias

### Objetivos de este Notebook:
- Explorar datos del segmento D (5K muestras vs 200K del segmento A)
- Analizar características del scoring crediticio de corto plazo
- Identificar patrones y distribuciones en datos financieros
- Preparar datos para generación sintética
- Establecer baseline de performance

### Contenido:
1. Configuración del entorno y librerías
2. Carga de datos (UCI + datos internos)
3. Análisis exploratorio del segmento D
4. Comparación con segmento A (referencia)
5. Análisis de variables de scoring crediticio
6. Preparación para generación de datos sintéticos


In [1]:
# Importación de librerías y configuración
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import yaml
import logging

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Cargar configuración
with open('../configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("✅ Librerías importadas correctamente")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")
print(f"⚙️ Configuración cargada: {len(config)} secciones")


✅ Librerías importadas correctamente
📊 Pandas version: 2.2.3
🔢 NumPy version: 1.26.4
📈 Matplotlib version: 3.10.1
🎨 Seaborn version: 0.13.2
⚙️ Configuración cargada: 8 secciones


In [2]:
# Importar módulos del proyecto
from src.data.data_loader import DataLoader
from src.data.data_preprocessor import DataPreprocessor
from src.data.data_splitter import DataSplitter
from src.data.data_validator import DataValidator

# Inicializar componentes
data_loader = DataLoader(config)
data_preprocessor = DataPreprocessor(config)
data_splitter = DataSplitter(config)
data_validator = DataValidator(config)

print("✅ Módulos del proyecto importados correctamente")
print("📁 DataLoader inicializado")
print("🔧 DataPreprocessor inicializado")
print("✂️ DataSplitter inicializado")
print("✅ DataValidator inicializado")


2025-10-17 13:25:21,585 - INFO - DataSplitter inicializado:
2025-10-17 13:25:21,585 - INFO -   Train: 0.6, Validation: 0.2, Test: 0.2
2025-10-17 13:25:21,585 - INFO -   Synthetic validation: 0.15
2025-10-17 13:25:21,585 - INFO -   CV folds: 5


✅ Módulos del proyecto importados correctamente
📁 DataLoader inicializado
🔧 DataPreprocessor inicializado
✂️ DataSplitter inicializado
✅ DataValidator inicializado


## 2. Carga de Datos de Crédito

Vamos a cargar datasets de crédito desde UCI Repository para simular el análisis del segmento D. Estos datasets nos permitirán:

1. **German Credit Data**: Dataset clásico de scoring crediticio
2. **Australian Credit Approval**: Datos de aprobación de crédito
3. **Credit Card Default**: Datos de default de tarjetas de crédito

Estos datasets nos servirán como proxy para entender las características del segmento D.


In [3]:
# Cargar datasets de crédito desde UCI
print("🔄 Cargando datasets de crédito desde UCI Repository...")

# Cargar German Credit Data (ID: 144)
try:
    german_features, german_targets = data_loader.load_uci_dataset(144, "german_credit")
    german_data = pd.concat([german_features, german_targets], axis=1)
    print(f"✅ German Credit Data cargado: {german_data.shape}")
    print(f"   Features: {german_features.shape[1]}")
    print(f"   Target distribution: {german_targets.iloc[:, 0].value_counts().to_dict()}")
except Exception as e:
    print(f"❌ Error cargando German Credit: {e}")
    german_data = None

# Cargar Australian Credit Approval (ID: 45)
try:
    australian_features, australian_targets = data_loader.load_uci_dataset(45, "australian_credit")
    australian_data = pd.concat([australian_features, australian_targets], axis=1)
    print(f"✅ Australian Credit Data cargado: {australian_data.shape}")
    print(f"   Features: {australian_features.shape[1]}")
    print(f"   Target distribution: {australian_targets.iloc[:, 0].value_counts().to_dict()}")
except Exception as e:
    print(f"❌ Error cargando Australian Credit: {e}")
    australian_data = None

# Cargar Credit Card Default (ID: 300)
try:
    default_features, default_targets = data_loader.load_uci_dataset(300, "credit_card_default")
    default_data = pd.concat([default_features, default_targets], axis=1)
    print(f"✅ Credit Card Default Data cargado: {default_data.shape}")
    print(f"   Features: {default_features.shape[1]}")
    print(f"   Target distribution: {default_targets.iloc[:, 0].value_counts().to_dict()}")
except Exception as e:
    print(f"❌ Error cargando Credit Card Default: {e}")
    default_data = None

print("\n📊 Resumen de datasets cargados:")
datasets_info = {
    'German Credit': german_data.shape if german_data is not None else "No disponible",
    'Australian Credit': australian_data.shape if australian_data is not None else "No disponible", 
    'Credit Card Default': default_data.shape if default_data is not None else "No disponible"
}

for name, shape in datasets_info.items():
    print(f"   {name}: {shape}")


2025-10-17 13:25:40,397 - INFO - Cargando dataset UCI ID: 144


🔄 Cargando datasets de crédito desde UCI Repository...


2025-10-17 13:25:42,265 - INFO - Dataset cargado: 1000 filas, 20 features
2025-10-17 13:25:42,265 - INFO - Target shape: (1000, 1)
2025-10-17 13:25:42,287 - INFO - Cargando dataset UCI ID: 45


✅ German Credit Data cargado: (1000, 21)
   Features: 20
   Target distribution: {1: 700, 2: 300}


2025-10-17 13:25:43,867 - INFO - Dataset cargado: 303 filas, 13 features
2025-10-17 13:25:43,871 - INFO - Target shape: (303, 1)
2025-10-17 13:25:43,873 - INFO - Cargando dataset UCI ID: 300


✅ Australian Credit Data cargado: (303, 14)
   Features: 13
   Target distribution: {0: 164, 1: 55, 2: 36, 3: 35, 4: 13}


2025-10-17 13:25:45,799 - INFO - Dataset cargado: 943 filas, 42 features
2025-10-17 13:25:45,799 - INFO - Target shape: (943, 1)


✅ Credit Card Default Data cargado: (943, 43)
   Features: 42
   Target distribution: {0: 478, 1: 465}

📊 Resumen de datasets cargados:
   German Credit: (1000, 21)
   Australian Credit: (303, 14)
   Credit Card Default: (943, 43)


In [4]:
# Seleccionar dataset principal para análisis (German Credit como proxy del segmento D)
if german_data is not None:
    # Usar German Credit como dataset principal
    segment_d_data = german_data.copy()
    segment_d_features = german_features.copy()
    segment_d_targets = german_targets.copy()
    
    print("🎯 Usando German Credit Data como proxy del Segmento D")
    print(f"   Tamaño: {segment_d_data.shape[0]} muestras")
    print(f"   Features: {segment_d_data.shape[1] - 1}")
    print(f"   Target: {segment_d_targets.columns[0]}")
    
    # Mostrar primeras filas
    print("\n📖 Primeras 5 filas del dataset:")
    display(segment_d_data.head())
    
    # Información básica del dataset
    print("\n📊 Información del dataset:")
    print(segment_d_data.info())
    
else:
    print("❌ No se pudo cargar el dataset principal")
    segment_d_data = None


🎯 Usando German Credit Data como proxy del Segmento D
   Tamaño: 1000 muestras
   Features: 20
   Target: class

📖 Primeras 5 filas del dataset:


,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,class
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,2



📊 Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Attribute1   1000 non-null   object
 1   Attribute2   1000 non-null   int64 
 2   Attribute3   1000 non-null   object
 3   Attribute4   1000 non-null   object
 4   Attribute5   1000 non-null   int64 
 5   Attribute6   1000 non-null   object
 6   Attribute7   1000 non-null   object
 7   Attribute8   1000 non-null   int64 
 8   Attribute9   1000 non-null   object
 9   Attribute10  1000 non-null   object
 10  Attribute11  1000 non-null   int64 
 11  Attribute12  1000 non-null   object
 12  Attribute13  1000 non-null   int64 
 13  Attribute14  1000 non-null   object
 14  Attribute15  1000 non-null   object
 15  Attribute16  1000 non-null   int64 
 16  Attribute17  1000 non-null   object
 17  Attribute18  1000 non-null   int64 
 18  Attribute19  1000 non-null   object
 19  

## 3. Análisis Exploratorio del Segmento D

Ahora vamos a realizar un análisis exploratorio completo del dataset que simula el segmento D. Este análisis incluirá:

1. **Análisis de calidad de datos**
2. **Estadísticas descriptivas**
3. **Análisis de distribuciones**
4. **Análisis de correlaciones**
5. **Identificación de patrones**


In [5]:
# Análisis de calidad de datos
if segment_d_data is not None:
    print("🔍 ANÁLISIS DE CALIDAD DE DATOS - SEGMENTO D")
    print("=" * 60)
    
    # Validar calidad de datos
    quality_results = data_validator.validate_data_quality(segment_d_data, target_col='class')
    
    # Mostrar resumen de calidad
    print("\n📊 RESUMEN DE CALIDAD:")
    print(f"   Shape: {quality_results['basic_info']['shape']}")
    print(f"   Memory usage: {quality_results['basic_info']['memory_usage'] / 1024**2:.2f} MB")
    
    # Valores faltantes
    missing_summary = quality_results['missing_values']['summary']
    print(f"   Missing values: {missing_summary['total_missing']} ({missing_summary['total_percentage']:.2f}%)")
    
    # Duplicados
    duplicates = quality_results['duplicates']
    print(f"   Duplicates: {duplicates['count']} ({duplicates['percentage']:.2f}%)")
    
    # Tipos de datos
    data_types = quality_results['data_types']
    numeric_cols = [col for col, info in data_types.items() if 'int' in info['dtype'] or 'float' in info['dtype']]
    categorical_cols = [col for col, info in data_types.items() if 'object' in info['dtype']]
    
    print(f"   Numeric columns: {len(numeric_cols)}")
    print(f"   Categorical columns: {len(categorical_cols)}")
    
    print("\n✅ Análisis de calidad completado")
else:
    print("❌ No hay datos disponibles para análisis de calidad")


2025-10-17 13:26:09,233 - INFO - Validando calidad de datos
2025-10-17 13:26:09,247 - INFO -   Shape: (1000, 21)
2025-10-17 13:26:09,247 - INFO -   Memory usage: 0.81 MB
2025-10-17 13:26:09,258 - INFO -   Missing values: 0 (0.00%)
2025-10-17 13:26:09,264 - INFO -   Duplicates: 0 (0.00%)
2025-10-17 13:26:09,271 - INFO -   Data types checked for 21 columns
2025-10-17 13:26:09,292 - INFO -   Outliers checked for 8 numeric columns
2025-10-17 13:26:09,314 - INFO -   Correlations checked


🔍 ANÁLISIS DE CALIDAD DE DATOS - SEGMENTO D

📊 RESUMEN DE CALIDAD:
   Shape: (1000, 21)
   Memory usage: 0.81 MB
   Missing values: 0 (0.00%)
   Duplicates: 0 (0.00%)
   Numeric columns: 8
   Categorical columns: 13

✅ Análisis de calidad completado
